<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Simple Decision Prompts

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using GenAI prompt engineering  
* Creating custom AI personas with a system decision prompt  
* Generating decision outputs using easier to consume formats (Markdown)     

Prompts are the fundemental building block to getting the proper responses from AI models. This module demonstrates how to use common prompt engineering techniques while using Semantic Kernel. If you've ever used ChatGPT or Microsoft Copilot, you're already familiar with prompting. Given a prompt instruction, an LLM will attempt to predict the most likely response. With Semantic Kernel, you can craft applications, services & APIs that execute prompts.  

For more information on using prompts with Semantic Kernel, visit: https://learn.microsoft.com/en-us/semantic-kernel/prompts/your-first-prompt  

The process of carefully crafting questions or instructions for AI is called **Prompt Engineering**. Prompt Engineering involves designing and refining input prompts—text or questions—so that the AI produces responses that are more relevant, useful, or accurate. The goal of prompt engineering is to "tune" the prompt in a way that maximizes the quality and clarity of the AI's response, often using specific wording, context, or examples within the prompt itself.

Decision Intelligence and Prompt Engineering techniques can be combined to created "Generative AI Decision Intellignce"! This can leverage GenAI models to reason over complex tasks, gather intelligence for decisions, recommend decisions, communicate decisions etc. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Json, 9.0.0

In [2]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.32.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Installed Packages Microsoft.SemanticKernel, 1.32.0

Using Azure OpenAI Service


### Step 2 - Execute a Decision Intelligence Prompt 

Many LLMs and even SLMs have been trained on knowledge that includes decision frameworks & processes. This makes LLMs great assistants to:
* Provide proper Decision Frames
* Gather Intelligence (information) in order to make a decision
* Recommend Decision Frameworks to make a high-quality decision
* Provide feedback to evaluate a Decision

Once the Semantic Kernel instance is built, it is ready to intake prompt instructions. In the prompt below, the Semantic Kernel is instructed to provide examples of decision frameworks "trained into" the knowledge of the configured AI model.  

In [3]:
// A new prompt to help with decision-making frameworks
var simpleDecisionPrompt = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.
""";

var simplePromptResponse = await semanticKernel.InvokePromptAsync(simpleDecisionPrompt);
Console.WriteLine(simplePromptResponse.GetValue<string>());

Certainly! Here are five decision-making frameworks that can enhance the quality of decisions along with brief descriptions of how each supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: SWOT analysis is a strategic planning framework used to identify the internal and external factors that can impact the success of a project or organization.
   - **Supportive Elements**:
     - **Strengths and Weaknesses**: Highlight internal factors, helping organizations leverage strengths and address weaknesses.
     - **Opportunities and Threats**: Focus on external factors that can be exploited or mitigated.
   - **Application**: Useful in strategic decision-making, helping to develop a well-rounded understanding of potential pathways and considerations.

2. **Cost-Benefit Analysis (CBA)**
   - **Description**: This framework involves comparing the costs and benefits associated with a decision to determine its overall

### Step 3 - Execute a Decision Intelligence Prompt with Streaming

Semantic Kernel supports prompt response streaming when invoking the prompt. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

> Note: An average human can read between 25-40 tokens / second.  

In [4]:
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt))
{
   Console.Write(streamChunk);
}

Certainly! Here are five decision-making frameworks that can enhance the quality of decisions along with brief descriptions of how each framework supports better analysis and reasoning:

1. **SWOT Analysis (Strengths, Weaknesses, Opportunities, Threats)**
   - **Description**: This framework helps in understanding both internal and external factors that impact a decision.
   - **Support for Better Analysis**: By systematically evaluating the positives (strengths and opportunities) and the negatives (weaknesses and threats), decision-makers can create balanced strategies. It facilitates a thorough analysis of the situational environment before making a decision.

2. **The Decision Matrix (Weighted Scoring Model)**
   - **Description**: This framework involves listing all possible options and evaluating each against a set of predefined criteria, each weighted according to its importance.
   - **Support for Better Analysis**: By quantifying and comparing the different aspects of each opti

### Step 4 - Execute a Semantic Kernel Decision Prompt with Improved Formatting  

Generative AI models have an inherent ability to not only provide decision reasoning analysis, but also format the output in a desired format. This could be as simple as instructing the Generative AI model to format the decision as a single sentence, paragraphs or lists. However more sophisticated output generations can be instructed. For example, the GenAI model can ouput Markdown or even extract information and fill in a desired schema (JSON). Specifically for Decision Intelligence, you can ask the GenAI models to apply decision communication frameworks to the generation as well. 

Execute the simple decision prompt below with Markdown formatting ouptut. This table can now be rendered in a Markdown document for easy human comprehension. Markdown tables and other formats can be used on web sites, document, programming code etc. Even Generative AI models understand Markdown format, which can not only be used for output but inside input prompts.  

In [6]:
// New prompt to help with decision-making frameworks using Markdown formatting.
var simpleDecisionPromptWithMarkdownFormat = """
Identify and list 5 decision-making frameworks that can enhance the quality of decisions. 
Briefly describe how each framework supports better analysis and reasoning in different decision-making scenarios.

Format the response using only a Markdown table. Do not enclose the table in triple backticks.
""";

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPromptWithMarkdownFormat))
{
   Console.Write(streamChunk);
}

| Framework                  | Description                                                                                                                                                                                |
|----------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis              | Identifies Strengths, Weaknesses, Opportunities, and Threats to evaluate the internal and external factors influencing a decision. Helps in strategic planning by organizing key data visually.     |
| Decision Matrix            | A grid that evaluates and prioritizes a list of options against a set of criteria. Weighted scores provide a clear comparison, aiding in choosing the most suitable option based on quantifiable data. |
| Cost-Benefit Analysis      | Compares the estimated costs against potential benefits to deter

Example formatted Markdown Table output: 

| Framework                  | Description                                                                                                                                                                                                                                                                                                                                |
|----------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SWOT Analysis              | SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis helps identify internal strengths and weaknesses, as well as external opportunities and threats. This framework is useful for strategic planning and risk management by providing a structured way to think about the pros and cons of different choices.                                         |
| Decision Matrix Analysis   | Also known as a prioritization matrix, this framework helps evaluate and prioritize different options by scoring them against a set of criteria. It is particularly useful in situations where many factors are at play, ensuring a comprehensive evaluation of all available options.                                                                    |
| Cost-Benefit Analysis      | This quantitative approach assesses the financial impacts of different decisions by comparing the expected costs and benefits. It supports better economic reasoning by making explicit the expected monetary outcomes of various alternatives and helps in choosing the option with the highest net benefit.                                        |
| The OODA Loop              | The OODA Loop (Observe, Orient, Decide, Act) is a decision-making process that emphasizes continuous feedback and learning. It is especially effective in rapidly changing environments, such as military strategy or business competition, as it promotes agility and adaptability in decision-making.                                            |
| The Delphi Method          | This structured communication technique uses a panel of experts who answer questionnaires in multiple rounds. After each round, a facilitator provides an anonymous summary of the experts' forecasts. This process helps in arriving at a more rounded and unbiased consensus, particularly useful in complex problem-solving and forecasting.            |

### Step 5 - Execute a Decision Intelligence Prompt with a Custom Execution Configuration

Semantic Kernel supports the configuration of prompt execution. The typical OpenAI and Azure OpenAI settings are exposed as configuration options that provide a different prompt experience. Try changing the MaxTokens, Tempature or other settings.

> Note: The supported settings are dependendent on the API plus the specific model version. For example, a model paired with an older API may not expose all the configuration settings available. Conversely, a new preview model may not have all the settings available until it becomes generally available.  

In [13]:
// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

Certainly! Here are five decision-making frameworks that are easy to understand and can be applied to various daily situations:

1. **Pros and Cons List**:
   - **How it works**: Write down the positive aspects (pros) and negative aspects (cons) of each option.
   - **Application**: This framework helps to visually compare the benefits and drawbacks of each choice, making it easier to see which option might be the best overall.
   - **Example**: Deciding whether to take a new job offer or stay at your current job.

2. **The Eisenhower Matrix**:
   - **How it works**: Categorize tasks into four quadrants based on urgency and importance: 
     1. Urgent and important
     2. Important but not urgent
     3. Urgent but not important
     4. Neither urgent nor important
   - **Application**: This helps prioritize tasks and focus on what truly matters.
   - **Example**: Managing daily tasks and deciding what to tackle first.

3. **SWOT Analysis**:
   - **How it works**: Analyze the Strength

### Step 6 - Execute a Decision Intelligence Prompt with a System Prompt (Custom AI Persona)

When building Decison Intelligence prompts, all the typical best practices of prompt engineering apply when using Semantic Kernel.  Such as: 
* Make the prompt more specific
* Add structure to the output with formatting
* Provide examples with few-shot prompting
* Instruct the AI what to do to avoid doing something wrong
* Provide context (additional information) to the AI
* Using Roles in chat completion prompts
* Give your AI words of encouragement  

A key best practice is to provide a common behavior across all the LLM interactions in a system prompt. This system prompt is passed in on every single call the LLM with Semantic Kernel. By passing the same (or similar) system prompt with every prompt gives your Generative AI system a common persona. This common persona is the foundational building block of building AI agents; where the desired behavior is to have each agent have a desired persona/behavior every time you interact with that agent.

Execute the cell below with a dynamic prompt template. Notice the different behavior of the output for decision frameworks. Based on the new system prompt instructions, the decision framework responses are much more robust with decision intelligence information.

In [14]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
var simpleDecisionPrompt = """
Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply in different scenarios.
""";

// How the prompt looks like to the LLM
var simpleDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {simpleDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 500, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

Console.WriteLine("PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);
Console.WriteLine(simpleDecisionPromptTemplate);
Console.WriteLine(string.Empty);
Console.WriteLine("----------------------------------------");
Console.WriteLine(string.Empty);
Console.WriteLine("RESPONSE FROM THE CHAT COMPLETION SERVICE...");
Console.WriteLine(string.Empty);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(simpleDecisionPrompt, kernelArguments))
{
   Console.Write(streamChunk);
}

PROMPT TO SEND TO THE CHAT COMPLETION SERVICE...

System Prompt: You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.

Request from the user: Recommend the top 5 decision frameworks that can be used for daily situations to make various decisions.
These frameworks should be very easy to understand and apply in different scenarios.

----------------------------------------

RESPONSE FROM THE CHAT COMPLETION SERVICE...

Certainly! Here are five decision frameworks that are easy to understand and can be applied to various daily situations:

### 1. **Pros and Cons List**
**Description:** This is one of the simplest and most widely used decision-making frameworks. It involves listing the advantages (pros) and disadvantages (cons) of each option.

**Steps:**
1. Identify the decision to be made.
2. List all poss

### Step 7 - Decision Scenario with a System Prompt (Custom AI Persona)

In this step a scenario will be introduced requiring analysis and a recommendation performed by Artificial Intelligence. The full Decision Intelligence framework will not be used, rather a simple request for Artificial Intelligene to recommend a path forward (decision recommedation) for the human user to ultimately make the final decision.  

**Decision Scenario:** Your high school daughter Alex is deciding whether to enroll directly in a four-year university or start at a community college to earn an associate degree first. These are the main decision factors: financial, career uncertainty, academic consistency and future impact. You have all the decision factor details available. You are looking for an impartial (non-family) recommendation. Can Artificial Intelligence help Alex decide? 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-SimpleDecision-College.png">

In [22]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. 
Provide structured, logical, and comprehensive decision advice.
""";
// Provide a description of the decision scenario and the desired output 
// Provide detailed decision scenario considerations and information about Alex (the decision-maker) 
var scenarioDecisionPrompt = """
Imagine you Are advising a daughter named Alex who is deciding whether to enroll directly in a well-regarded four-year 
university or start at a community college to earn an associate degree first. 

Make a single recommendation based on the following decision factor details. 
Output the recommedation in a Markdown table format. 

Financial Considerations:
A four-year university will cost significantly more in tuition, housing, and related expenses 
(estimated $50,000-$60,000 per year). A two-year associate program at a local community college could 
save substantial money (estimated $3,000-$5,000 per year), but Alex may have to transfer to a 
four-year institution later to complete a bachelor’s degree. The family can afford the four-year university, 
with some loans, but the cost is only a medium concern. 

Career and Major Uncertainty:
Alex is not entirely sure what she wants to major in. She is torn between business, psychology, and 
possibly something in the arts. She worries that if she starts at the four-year university, 
she might switch majors and incur extra time and cost. On the other hand, 
community college might give her space to explore options, 
but transferring could mean adjusting to a new campus and curriculum midway through.

Academic Consistency and Networking:
Going straight to the four-year university would allow Alex to forge early relationships with professors, 
join campus groups, and potentially secure internships or research opportunities. Starting at community college might 
delay those opportunities, but it could also let her explore different fields at a lower cost. 
She might miss out on the “full campus” experience early on, but transferring later means she could 
still build connections, just on a different timeline. Alex is not overly concerned about the social aspect of college.

Future Impact:
Alex wants a solid professional network and relevant experience when she graduates. She is also concerned 
about taking on significant student loan debt. The decision affects not only her immediate academic path but 
also her long-term financial stability and career prospects. Given these factors, provide a reasoned analysis of 
whether Alex should attend the four-year university right away or begin with an associate degree program at a 
local community college.
""";

// How the prompt looks like to the LLM
var scenarioDecisionPromptTemplate = $"""
System Prompt: {systemPromptForDecisions}

Request from the user: {scenarioDecisionPrompt}
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    ChatSystemPrompt = systemPromptForDecisions,
    MaxTokens = 750, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(scenarioDecisionPromptTemplate, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the decision factors provided, here is a structured recommendation for Alex:

| Decision Factor                | Four-Year University                        | Community College                              | Analysis and Recommendation                                                                 |
|--------------------------------|---------------------------------------------|------------------------------------------------|--------------------------------------------------------------------------------------------|
| **Financial Considerations**   | $50,000-$60,000 per year, family can afford with loans | $3,000-$5,000 per year, significant savings   | Community college offers substantial cost savings, reducing the need for loans and financial burden. |
| **Career and Major Uncertainty**| Risk of extra time and cost if switching majors | Flexibility to explore different fields       | Community college provides a lower-cost environment for exploring majors, reducing the ri